In [1]:
# Import libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Get the current working directory
pwd= os.getcwd()

In [3]:
df_domainactive = pd.read_csv(pwd + "/domain12july.csv")
df_taboola = pd.read_excel(pwd + "/Taboola_Spends_July12.xlsx")

#df_managerslist = pd.read_excel(pwd + "/ManagersList.xlsx")
#df_campmanagerslist = pd.read_excel(pwd + "/CampManagersList.xlsx")

In [4]:
# drop zero impressions rows
drop1 = df_taboola[ (df_taboola['Viewable Impressions'] == 0)].index
df_taboola.drop(drop1 , inplace=True)
df_taboola.rename(columns={'ID': 'campaign_number'}, inplace=True)

# Convert 'Spent' and 'Actual CPA' columns to float
df_taboola['Spent'] = df_taboola['Spent'].astype(float)
df_taboola['Actual CPA'] = df_taboola['Actual CPA'].astype(float)

# Convert 'Viewable Impressions', 'Clicks', and 'Conversions' columns to integer
df_taboola['Viewable Impressions'] = df_taboola['Viewable Impressions'].astype(int)
df_taboola['Clicks'] = df_taboola['Clicks'].astype(int)
df_taboola['Conversions'] = df_taboola['Conversions'].astype(int)


df_taboola

In [5]:
#df_taboola['campaign_number'] = df_taboola['campaign_number'].astype(str)

In [6]:
# Group the data by campaign_number and calculate the sum of publisher_revenue_amount and revenue_clicks
#grouped_data = df_domainactive.groupby('campaign_number')['publisher_revenue_amount', 'revenue_clicks'].sum().reset_index()
grouped_data = df_domainactive.groupby('campaign_number')[['publisher_revenue_amount', 'revenue_clicks','tracked_visitors']].sum().reset_index()

In [7]:
drop2 = grouped_data[ (grouped_data['publisher_revenue_amount'] == 0) & (grouped_data['revenue_clicks'] ==0) ].index
grouped_data.drop(drop2 , inplace=True)

In [8]:
#merged_data = df_taboola.merge(grouped_data, left_on='campaign_number', right_on='ID')
#merged_data = pd.merge(grouped_data, df_taboola, on=['campaign_number', 'ID'])
#merged_data = pd.merge(df_taboola, grouped_data, on=['campaign_number'], how='left')
merged_data = pd.merge(df_taboola, grouped_data, on=['campaign_number'], how='left')

In [9]:
#merged_data = grouped_data.merge(df_taboola, on=['campaign_number', 'ID'])


In [10]:
df_blankids = df_domainactive[df_domainactive['campaign_number'].isna()]

# Perform aggregations on 'publisher_revenue_amount' and 'revenue_clicks' columns
df_blankids = df_blankids.groupby('campaign__name').agg({
    'publisher_revenue_amount': 'sum',
    'revenue_clicks': 'sum',
    'tracked_visitors':'sum'
}).reset_index()

In [11]:
#NEW
df_domainactive['campaign_number'] = df_domainactive['campaign_number'].fillna(0).astype(int)

In [12]:

#df_category = df_domainactive.groupby('campaign_number')[['campaign__category']].agg(' '.join).reset_index()

In [13]:
#drop duplicate values before performing the grouping and aggregation.

df_domainactive_unique = df_domainactive.drop_duplicates(subset=['campaign_number', 'campaign__category'])
df_category = df_domainactive_unique.groupby('campaign_number')['campaign__category'].agg(' '.join).reset_index()


In [14]:
#Remove null rows
df_blankids = df_blankids[(df_blankids['publisher_revenue_amount'] != 0) | (df_blankids['revenue_clicks'] != 0)]


# Rename
df_blankids.rename(columns={'campaign__name': 'Name'}, inplace=True)

In [15]:
# Add the rows from df_blankids to merged_data based on 'Name' column
finalmerged_data = pd.concat([merged_data, df_blankids], ignore_index=True)

In [16]:
finalmerged_data_1 = finalmerged_data.merge(df_category, on='campaign_number', how='left')

In [17]:
finalmerged_data_1

,Name,campaign_number,Spent,Viewable Impressions,Clicks,Conversions,Actual CPA,Conversion Rate,Actual CPC,vCTR,vCPM,Value,ROAS,Account,publisher_revenue_amount,revenue_clicks,tracked_visitors,campaign__category
0,Copy of Copy of T4-DA | USA | Veterans Assista...,26223294.0,603.48,85605.0,1275.0,241.0,2.504,0.1890,0.473,0.0149,7.05,0.0,0.0,T4,764.34,221.0,499.0,Veterans Assistance
1,Copy of T4-DA | USA | Personal Injury Attorney...,25601703.0,301.37,55012.0,936.0,156.0,1.932,0.1667,0.322,0.0170,5.48,0.0,0.0,T4,349.87,168.0,504.0,Personal Injury Attorneys
2,T4-DA | USA | High School Diploma | Mobile #16...,25140786.0,295.19,48882.0,769.0,104.0,2.838,0.1352,0.384,0.0157,6.04,0.0,0.0,T4,186.59,100.0,237.0,High School Diploma
3,T4-DA | USA | Migraine-Treatment-Mobile | #190...,25194183.0,293.77,41133.0,909.0,158.0,1.859,0.1738,0.323,0.0221,7.14,0.0,0.0,T4,269.20,140.0,452.0,Migraine treatment
4,T4-DA | USA | Personal Injury Attorneys | Mobi...,25542871.0,285.03,65825.0,656.0,167.0,1.707,0.2546,0.435,0.0100,4.33,0.0,0.0,T4,302.12,142.0,391.0,Personal Injury Attorneys
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,Personal Injury Attorneys_P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.41,23.0,179.0,NaN
216,Plumbing Jobs_Z1_Taboola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.08,2.0,1.0,NaN
217,Student Loans - T5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.79,18.0,75.0,NaN
218,Tax_Relief_P_09052023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.15,2.0,44.0,NaN


In [18]:
# Extract the string after the hyphen from the left in the "Name" column and store it in "username" column
#merged_data['username'] = merged_data['Name'].str.split('-').str[1].str.strip()
finalmerged_data_1['username'] = finalmerged_data['Name'].str.rsplit('-', n=1).str[-1].str.strip()

In [19]:
# Create a pivot table to view the grouped data

finalmerged_data_1.pivot_table(['publisher_revenue_amount','Spent'], index='username', aggfunc='sum', margins=True).sort_values(by='publisher_revenue_amount', ascending=False)
#pivot_data_sorted = pivot_data.sort_values(by='publisher_revenue_amount', ascending=False)

,Spent,publisher_revenue_amount
username,,
All,11046.75,12817.62
nayan,4640.18,6094.46
ruchitajain,4048.47,4106.43
Sparsh,1906.76,1983.18
Mansi,365.78,322.49
pawansingh,291.90,307.29
Malpractice Attorneys T5_RS_,0.00,221.14
Veterans Assistance T5_RS_,0.00,178.03
T5,0.00,150.01


In [20]:
finalmerged_data_1.to_excel("Taboola_DA_July_12.xlsx", index=False)